In [ ]:
import subprocess
import os
print('Current conda environment:', os.environ['CONDA_DEFAULT_ENV'])
os.environ['TOKENIZERS_PARALLELISM'] = "false"

cwd = os.getcwd()
print(cwd)

import warnings
warnings.filterwarnings('ignore')

import random
random.seed(42)

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc = {'figure.figsize':(15,8)})

from crem.crem import grow_mol, mutate_mol
crem_db = '../crem_db/crem_db2.5.db'

import mols2grid

from rdkit import Chem
from rdkit.Chem import AllChem, rdFingerprintGenerator, CanonSmiles, Draw, MolFromSmiles, PandasTools
from rdkit.Chem.rdmolops import RDKFingerprint
from rdkit.Chem.Draw import MolsToGridImage
from rdkit import DataStructs
from rdkit.Chem.rdFMCS import FindMCS
from rdkit.DataStructs.cDataStructs import BulkTanimotoSimilarity
import useful_rdkit_utils as uru

import prolif as plf

import safe as sf
import datamol as dm

import mols2grid

from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, classification_report
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import torch
import itertools 
from coati.generative.coati_purifications import embed_smiles
from coati.models.io.coati import load_e3gnn_smiles_clip_e2e
from coati.models.simple_coati2.io import load_coati2


from rdkit.Chem.Scaffolds import MurckoScaffold
from rdkit.Chem.Scaffolds import rdScaffoldNetwork
from collections import defaultdict
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from collections import defaultdict

### Visualizations Set Up

In [ ]:
#Seaborn settings for visualizations

rc = {
    "axes.facecolor": "#f7f9fc",
    "figure.facecolor": "#f7f9fc",
    "axes.edgecolor": "#000000",
    "grid.color": "#EBEBE7",
    "font.family": "serif",
    "axes.labelcolor": "#000000",
    "xtick.color": "#000000",
    "ytick.color": "#000000",
    "grid.alpha": 0.4
}
/
default_palette = 'tab10'

sns.set(rc=rc)

# Initial Set Up

Load initial ligand

In [ ]:
pdb = '2zdt'

In [ ]:
initial_mol = Chem.MolFromMolFile(f"data/docking/{pdb}_ligand.sdf")
initial = Chem.MolToSmiles(initial_mol)

MolsToGridImage([Chem.MolFromSmiles(initial)], subImgSize=(600, 600))

In [ ]:
initial

Obtain Molecular Formula of Initial Ligand

In [ ]:
# Convert SMILES to a molecule object
mol = Chem.MolFromSmiles(initial)

# Get the molecular formula
formula = Chem.rdMolDescriptors.CalcMolFormula(mol)

formula

# Interaction Fingerprints

### Interaction fingerprint for reference molecule

In [ ]:
REF_MOL_FILEPATH = f"data/docking/{pdb}_ligand.sdf"
PDB_FILEPATH = f"data/docking/{pdb}.pdb"

fp = plf.Fingerprint()

mol = Chem.MolFromPDBFile(PDB_FILEPATH, removeHs=False)
prot = plf.Molecule(mol)
suppl = plf.sdf_supplier(REF_MOL_FILEPATH)
fp.run_from_iterable(suppl, prot, progress=True)
df_ifp = fp.to_dataframe()
df_ifp.columns = df_ifp.columns.droplevel(0)

In [ ]:
df_ifp

### Interaction fingerprint functions

ifp_similarity computes the intersection of IMFs between a generated molecule and a reference fragment

In [ ]:
def ifp_similarity(ref_mol_ifp, df_ifp, df):
    ## Rename columns
    df_ifp.columns = [' '.join(col) if isinstance(col, tuple) else col for col in df_ifp.columns]
    ref_mol_ifp.columns = [' '.join(col) if isinstance(col, tuple) else col for col in ref_mol_ifp.columns]
    

    intersections = []
    weighted_intersections = []

    #iterate over the rows
    for index, row in df_ifp.iterrows():
        count=0
        weighted_count = 0
        #iterate over all columns
        for col_name in df_ifp.columns:
            if col_name in ref_mol_ifp.columns and df_ifp[col_name][index]==ref_mol_ifp[col_name][0] and 'VdWContact' in col_name:
                count += 1
                weighted_count += 1
            elif col_name in ref_mol_ifp.columns and df_ifp[col_name][index]==ref_mol_ifp[col_name][0] and 'Hydrophobic' in col_name:
                count += 1
                weighted_count += 2
            elif col_name in ref_mol_ifp.columns and df_ifp[col_name][index]==ref_mol_ifp[col_name][0] and 'HBAcceptor' in col_name:
                count += 1
                weighted_count += 3
            elif col_name in ref_mol_ifp.columns and df_ifp[col_name][index]==ref_mol_ifp[col_name][0] and 'Anionic' in col_name or 'Cationic' in col_name:
                count += 1
                weighted_count += 4
        
        intersections.append(count)
        weighted_intersections.append(weighted_count)
                
    df['IFP Intersection'] = intersections
    df['Weighted IFP Intersection'] = weighted_intersections

    return df


compute_features counts total number of interactions, number of interactions per IMF and weighted number of interactions for generated molecule.

In [ ]:
def compute_features(df, ifp):

    cols = ifp.columns

    num_cols = len(ifp.columns)

    data = {'mol_id' : [],
            'num_interactions' : [],
            'weighted_interactions' : [],
            'num_VdW' : [],
            'num_hydrophobic' : [],
            'num_HBAcceptor' : [],
            'num_ionic' : []}

    for index, row in ifp.iterrows():

        weighted_interactions = 0
        num_VdW = 0
        num_hydrophobic = 0
        num_HBAcceptor = 0
        num_ionic = 0
        
        data['mol_id'].append(row['ID'][0])
        data['num_interactions'].append(row[:-1].sum())

        for value in cols:
            
            if value[1] == 'VdWContact':
                weighted_interactions += 1 * row[value]
                num_VdW += 1 * row[value]
            elif value[1] == 'Hydrophobic':
                weighted_interactions += 2 * row[value]
                num_hydrophobic += 1 * row[value]
            elif value[1] == 'HBAcceptor':
                weighted_interactions += 3 * row[value]
                num_HBAcceptor += 1 * row[value]
            elif value[1] == 'Anionic' or value[1] == 'Cationic':
                weighted_interactions += 4 * row[value]
                num_ionic += 1 * row[value]

        data['weighted_interactions'].append(weighted_interactions)
        data['num_VdW'].append(num_VdW)
        data['num_hydrophobic'].append(num_hydrophobic)
        data['num_HBAcceptor'].append(num_HBAcceptor)
        data['num_ionic'].append(num_ionic)

            

    features = pd.DataFrame(data)

    df = df.merge(features[['mol_id', 'num_interactions', 'weighted_interactions', 'num_VdW', 'num_hydrophobic', 'num_HBAcceptor', 'num_ionic']], left_on='ID', right_on='mol_id', how='left')

    df = df.drop(['mol_id'], axis=1).sort_values(['Docking score'], ascending=True)

    df.dropna(axis=0, subset=['Docking score'], inplace=True)
    df['num_interactions'].fillna(0, inplace=True)
    df['weighted_interactions'].fillna(0, inplace=True)

    return df

In [ ]:
def visualize_fingerprint(ifp):

    sns.set(rc = {'figure.figsize':(15,8)})
    ax = sns.heatmap(ifp,cmap=sns.cm.rocket_r)
    ax.set_ylabel("Molecule")
    ax.set_xlabel("Protein Interaction")

    return ax

### Handling Duplicates Functions

Function will be used to remove duplicates produced by the same model

In [ ]:
def remove_repeats(df, inchi_names):
    """
    Removes rows with repeated 'inchi' and 'Model' values.

    Parameters:
    df (pd.DataFrame): Input DataFrame with columns 'inchi' and 'Model'
    inchi_names (list): List of inchi names to check for repeats

    Returns:
    pd.DataFrame: DataFrame with repeats removed
    """
    # Iterate over each inchi name in inchi_names
    for inchi_name in inchi_names:
        # Get the subset of the DataFrame for the current inchi_name
        df_inchi = df[df['inchi'] == inchi_name]
        
        # Find duplicates based on 'inchi' and 'Model' columns
        duplicates = df_inchi[df_inchi.duplicated(subset=['inchi', 'Model'], keep='first')]
        
        # Drop the duplicate rows from the original DataFrame
        df = df.drop(duplicates.index)
    
    return df

The following functions will be used to identify duplicates across different models

Not all molecules seemed to have a 3D conformers, so we make sure that they do with the ensure 3d conformers

In [ ]:
def ensure_3d_conformer(mol):
    """
    Ensure that the RDKit molecule object has a 3D conformation.
    
    Parameters:
    - mol: RDKit molecule object
    
    Returns:
    - mol: RDKit molecule object with a 3D conformation
    """
    if not mol.GetNumConformers():
        # Add hydrogens if not already present
        mol = Chem.AddHs(mol)
        
        # Generate 3D conformation
        result = AllChem.EmbedMolecule(mol)
        if result == -1:  # -1 means embedding failed
            raise ValueError("Failed to generate 3D conformation for the molecule.")
        
        # Optimize the 3D conformation
        AllChem.UFFOptimizeMolecule(mol)
        
    return mol


Following function calculates RMSD between two molecules

In [ ]:
def calculate_rmsd(mol1, mol2):
    """
    Calculate RMSD between two RDKit molecule objects with 3D conformations.
    
    Parameters:
    - mol1: RDKit molecule object
    - mol2: RDKit molecule object
    
    Returns:
    - RMSD value (float)
    """
    # Ensure both molecules have 3D conformers
    mol1 = ensure_3d_conformer(mol1)
    mol2 = ensure_3d_conformer(mol2)
    
    if not mol1.GetNumConformers() or not mol2.GetNumConformers():
        raise ValueError("One or both molecules could not be embedded or optimized.")

    # Calculate RMSD
    rms = AllChem.GetBestRMS(mol1, mol2)
    
    return rms


Detect duplicates based on same inchi and RMSD value of less than or equal to 3 (most molecules do and threshold can be adjusted)

In [ ]:
#Takes as input an repeated inchi value and returns a dataframe of molecules that have RMSD <=0.3 and their corresponding RoMOL
def duplicate_detector(df):
    
    duplicates_list=[]
    for a,b in itertools.combinations(df.ROMol,2):

        #Calculate average RMSD since it changes at every run
        RMSD_list=[]
        for _ in range(10):
            RMSD = calculate_rmsd(a,b)
            RMSD_list+=[RMSD]
        # Filter out non-numeric values
        RMSD_list = [num for num in RMSD_list if isinstance(num, (int, float))]

        # Calculate the average
        if RMSD_list:
            RMSD_average = sum(RMSD_list) / len(RMSD_list)
        else:
            RMSD_average = None

        rms = RMSD_average

        #Filter at 1 Amstrong
        if rms <=3:
            # Get SMILES String
            mol_a = df[df['ROMol'] == a].iloc[0,0]
            mol_b = df[df['ROMol'] == b].iloc[0,0]
            # Get model
            model_a_index = df[df['ROMol'] == a].index[0]
            model_a= df.at[model_a_index, 'Model']
            
            model_b_index = df[df['ROMol'] == b].index[0]
            model_b= df.at[model_b_index, 'Model']

            #Add ROMol
            duplicates_list.append((mol_a, model_a, a, mol_b, model_b, b, rms))
    column_names = ['Smiles_1', 'model_1', 'ROMol_1', 'Smiles_2', 'model_2', 'ROMol_2', 'Average_RMSD']
    df_new=pd.DataFrame(duplicates_list, columns=column_names)

    return df_new   

Function checks repeated molecules from all of the repeated inchi values in the dataset

In [ ]:
#Takes as input dataframe with all data and a list of repeated inchi names and returns a dataframe with duplicates
def process_inchi_names(model_df, inchi_names):
    """Process a list of InChI names and concatenate results."""
    all_results = []
    
    for inchi_name in inchi_names:
        # Filter the DataFrame for the current InChI name
        df_filtered = model_df.query(f'inchi == "{inchi_name}"')
        
        # Apply the duplicate_detector function
        df_duplicates = duplicate_detector(df_filtered)
        
        # Append result if it's not empty
        if not df_duplicates.empty:
            all_results.append(df_duplicates)
    
    # Concatenate all results into a single DataFrame, if there are any
    if all_results:
        final_df = pd.concat(all_results, ignore_index=True)
    else:
        # Return an empty DataFrame with the same columns if no results
        column_names = ['Smiles_1', 'model_1', 'ROMol_1', 'Smiles_2', 'model_2', 'ROMol_2', 'Average_RMSD']
        final_df = pd.DataFrame(columns=column_names)
    
    return final_df


The following function removes generated molecules that have more than one fragment

In [ ]:
def remove_broken_molecules(df):

    smiles = df['SMILES'].to_list()

    mols = [Chem.MolFromSmiles(smile) for smile in smiles]

    df['ROMol'] = mols

    df.dropna(subset=['ROMol'], inplace=True)

    df['num_frags'] = df.ROMol.apply(uru.count_fragments)

    df_single_frag = df.query("num_frags == 1").copy()

    return df_single_frag

# Generating Analogs


## REINVENT

In [ ]:
model = 'reinvent'

arg1 = '--model'
arg2 = '--sample'
arg3 = '--dock'
arg4 = '--pdb'

args = ['python3', 'generate_analogs.py',
        arg1, model,
        arg2, '200',
        arg3,
        arg4, pdb]

# Change directory to generate analogs with python script
%cd ..

subprocess.run(args,
               stdout=subprocess.DEVNULL,
               stderr=subprocess.STDOUT)
        
# Change directory back to that of the current notebook
%cd experiments

In [ ]:
DF_FILEPATH = f'experiments/data/{model}_dataframe.csv'
IFP_FILEPATH = f'experiments/data/{model}_ifp.csv'

df_reinvent = pd.read_csv(DF_FILEPATH, index_col=0)

ifp_reinvent = pd.read_csv(IFP_FILEPATH, header=[0, 1], index_col=0)

Compute metrics

In [ ]:
#Compute length to check that no molecules are being filtered by metric computation
len(df_reinvent)

In [ ]:
#number of IMFs
df_reinvent = compute_features(df_reinvent, ifp_reinvent)
# Compare IMFs to initial fragment
df_reinvent = ifp_similarity(df_ifp, ifp_reinvent, df_reinvent)

df_reinvent.drop(['Input_SMILES', 'Prior', 'Tanimoto'], axis=1, inplace=True)
df_reinvent['Model'] = model


### Handling Duplicates and Broken Molecules

In [ ]:
# Add a column with mol object information for each molecule
df_reinvent['ROMol']=[Chem.MolFromSmiles(x) for x in df_reinvent['SMILES'].values]
df_reinvent['inchi'] = df_reinvent.ROMol.apply(Chem.MolToInchiKey)

# Count repeated InChI
value_counts_reinvent=df_reinvent.inchi.value_counts()
filtered_counts_reinvent = value_counts_reinvent[value_counts_reinvent > 1]

# Get values for InChI
inchi_names_reinvent=filtered_counts_reinvent.index.tolist()
filtered_counts_reinvent


In [ ]:
df_reinvent = remove_repeats(df_reinvent, inchi_names_reinvent)
df_reinvent = remove_broken_molecules(df_reinvent)
df_reinvent

## CReM

In [ ]:
model = 'crem'

arg1 = '--model'
arg2 = '--sample'
arg3 = '--dock'
arg4 = '--pdb'

args = ['python3', 'generate_analogs.py',
        arg1, model,
        arg2, '200',
        arg3,
        arg4, pdb]

# Change directory to generate analogs with python script
%cd ..

subprocess.run(args,
               stdout=subprocess.DEVNULL,
               stderr=subprocess.STDOUT)
        
# Change directory back to that of the current notebook
%cd experiments

In [ ]:
DF_FILEPATH = f'experiments/data/{model}_dataframe.csv'
IFP_FILEPATH = f'experiments/data/{model}_ifp.csv'

df_crem = pd.read_csv(DF_FILEPATH, index_col=0)

ifp_crem = pd.read_csv(IFP_FILEPATH, header=[0, 1], index_col=0)

In [ ]:
#Compute length to check that no molecules are being filtered by metric computation
len(df_crem), len(ifp_crem)

Compute metrics

In [ ]:
#number of IMFs
df_crem = compute_features(df_crem, ifp_crem)
len(df_crem)

Remark: compute_features drops molecules that are not able to dock, i.e, do not have a docking score

In [ ]:
# Compare IMFs to initial fragment
df_crem = ifp_similarity(df_ifp, ifp_crem, df_crem)

In [ ]:
df_crem

### Handling Duplicates and Broken Molecules

In [ ]:
# Add a column with mol object information for each molecule
df_crem['ROMol']=[Chem.MolFromSmiles(x) for x in df_crem['SMILES'].values]
df_crem['inchi'] = df_crem.ROMol.apply(Chem.MolToInchiKey)

# Count repeated InChI
value_counts_crem=df_crem.inchi.value_counts()
filtered_counts_crem = value_counts_crem[value_counts_crem > 1]

# Get values for InChI
inchi_names_crem=filtered_counts_crem.index.tolist()
filtered_counts_crem


In [ ]:
df_crem=remove_repeats(df_crem, inchi_names_crem)
df_crem= remove_broken_molecules(df_crem)
df_crem

Create dataframe that contains all generated molecules

In [ ]:
model_df = pd.concat((df_reinvent, df_crem))


## Coati

In [ ]:
model = 'coati'

arg1 = '--model'
arg2 = '--sample'
arg3 = '--dock'
arg4 = '--pdb'

args = ['python3', 'generate_analogs.py',
        arg1, model,
        arg2, '200',
        arg3,
        arg4, pdb]

# Change directory to generate analogs with python script
%cd ..

subprocess.run(args,
               stdout=subprocess.DEVNULL,
               stderr=subprocess.STDOUT)
        
# Change directory back to that of the current notebook
%cd experiments

In [ ]:
DF_FILEPATH = f'experiments/data/{model}_dataframe.csv'
IFP_FILEPATH = f'experiments/data/{model}_ifp.csv'

df_coati = pd.read_csv(DF_FILEPATH, index_col=0)

ifp_coati = pd.read_csv(IFP_FILEPATH, header=[0, 1], index_col=0)

Compute metrics

In [ ]:
#number of IMFs
df_coati = compute_features(df_coati, ifp_coati)
df_coati = ifp_similarity(df_ifp, ifp_coati, df_coati)

In [ ]:
#Visualize df_coati
df_coati

### Handling Duplicates and Broken Molecules

In [ ]:
# Add a column with mol object information for each molecule
df_coati['ROMol']=[Chem.MolFromSmiles(x) for x in df_coati['SMILES'].values]
df_coati['inchi'] = df_coati.ROMol.apply(Chem.MolToInchiKey)

# Count repeated InChI
value_counts_coati=df_coati.inchi.value_counts()
filtered_counts_coati = value_counts_coati[value_counts_coati > 1]

#Get values of InChi
inchi_names_coati=filtered_counts_coati.index.tolist()
filtered_counts_coati


In [ ]:
df_coati = remove_repeats(df_coati, inchi_names_coati)
df_coati = remove_broken_molecules(df_coati)

In [ ]:
model_df = pd.concat((model_df, df_coati))

In [ ]:
model_df

## SAFE

In [ ]:
model = 'safe'

arg1 = '--model'
arg2 = '--sample'
arg3 = '--dock'
arg4 = '--pdb'

args = ['python3', 'generate_analogs.py',
        arg1, model,
        arg2, '200',
        arg3,
        arg4, pdb]

# Change directory to generate analogs with python script
%cd ..

subprocess.run(args,
               stdout=subprocess.DEVNULL,
               stderr=subprocess.STDOUT)
        
# Change directory back to that of the current notebook
%cd experiments

In [ ]:
DF_FILEPATH = f'experiments/data/{model}_dataframe.csv'
IFP_FILEPATH = f'data/{model}_ifp.csv'


df_safe = pd.read_csv(DF_FILEPATH, index_col=0)

ifp_safe = pd.read_csv(IFP_FILEPATH, header=[0, 1], index_col=0)

Compute metrics

In [ ]:
#Compute IMFs
df_safe = compute_features(df_safe, ifp_safe)
#Compare IMFs of reference molecule
df_safe = ifp_similarity(df_ifp, ifp_safe, df_safe)

In [ ]:
#visualize df
df_safe

### Handling Duplicates and Broken Molecules

In [ ]:
# Add a column with mol object information for each molecule
df_safe['ROMol']=[Chem.MolFromSmiles(x) for x in df_safe['SMILES'].values]
df_safe['inchi'] = df_safe.ROMol.apply(Chem.MolToInchiKey)

# Count repeated InChI
value_counts_safe=df_safe.inchi.value_counts()
filtered_counts_safe = value_counts_safe[value_counts_safe > 1]

# Get values for InChI
inchi_names_safe=filtered_counts_safe.index.tolist()



In [ ]:
df_safe = remove_repeats(df_safe, inchi_names_safe)
df_safe = remove_broken_molecules(df_safe)

In [ ]:
len(df_safe)

In [ ]:
model_df = pd.concat((model_df, df_safe))

In [ ]:
model_df

# Evaluating Metrics with MolScore

In [ ]:
from molscore import MolScore

In [ ]:
smiles = model_df['SMILES'].to_list()

In [ ]:
ms = MolScore(model_name='mol2mol', task_config='molscore/feature_selection.json')

scores = ms.score(smiles)

In [ ]:
file_path = 'experiments/data/goodness_scoring_data/2024_08_07_mol2mol_feature_selection_18_21_40/iterations/000001_scores.csv'  

molscore_df = pd.read_csv(file_path, index_col=0)

In [ ]:
molscore_df

# Correlation Analysis

## Set up

Remark: even though the indexes in model_df and df do not match, the information in each row does match

In [ ]:
model_df

In [ ]:
molscore_df

### Creating features dataframe

Define $X$

In [ ]:
# drop categorical features and redudant features

# Got rid of SA score and desc_PenLogP  because all entries are zero
# Got rid of QED as in ecompasses several other features

X = molscore_df.drop(['smiles', 'model', 'task', 'step',
            'batch_idx', 'absolute_time',
            'valid', 'valid_score', 'unique',
            'occurrences','dice_Cmpd1_Sim', 'tanimoto_Cmpd1_Sim', 
            'amean', 'filter',
            'score_time', 'raw_valid_score','desc_MolecularFormula', 'desc_SAscore', 'desc_PenLogP', 'desc_QED'], axis=1)

Adding new features

In [ ]:
X['ifp_intersection']=model_df['IFP Intersection'].values 
X['Docking score'] = model_df['Docking score'].values
X['num_interactions'] = model_df['num_interactions'].values

In [ ]:
# weighted_ifp_intersection = model_df['Weighted IFP Intersection'].values
# X['ifp_intersection']=model_df['IFP Intersection'].values 
# X['Weighted IFP Similarity'] = weighted_ifp_intersection / model_df['weighted_interactions'].values
# X['IFP Similarity'] = weighted_ifp_intersection / model_df['weighted_interactions'].values
# X['Docking score'] = model_df['Docking score'].values
# X['num_interactions'] = model_df['num_interactions'].values
# X['weighted_interactions'] = model_df['weighted_interactions'].values
# X['Interaction Weight Ratio'] = model_df['weighted_interactions'].values / model_df['num_interactions'].values

Renaming features

In [ ]:
molscore_df.columns

In [ ]:
column_names = {
    'desc_Bertz' : 'Synthetic Complexity',
    'interaction weight ratio' : 'Avg Interaction Strength',
    'Weighted IFP Similarity' : 'Weighted Interaction Similarity',
    'RAScore_pred_proba' : 'Synthetic Accessibility',
    'desc_NumHeteroatoms' : '# Heteroatoms',
    'desc_HeavyAtomMolWt': "Heavy Atom MolWt", 
    'desc_NumHAcceptors': '# HAcceptors', 
    'desc_NumHDonors':"#HDonors",
    'desc_NumRotatableBonds': '# Rotatable Bonds',
    'desc_NumAromaticRings': '# Aromatic Rings', 
    'desc_NumAliphaticRings': 'Number Aliphatic Rings', 
    'desc_RingCount': 'Ring Count',
    'desc_TPSA': 'TPSA', 
    'desc_FormalCharge': 'Formal Charge',
    'desc_CLogP': 'CLogP',
    'desc_MolWt': 'MolWt', 
    'desc_HeavyAtomCount': 'Heavy Atom Count',
    'desc_MaxConsecutiveRotatableBonds': 'Max Consecutive Rotatable Bonds',
    'tanimoto_Sim': 'Tanimoto Sim',
    'dice_Sim': 'Dice Sim',
    'desc_FlourineCount':'Fluorine Count'
    }

X.rename(columns=column_names, inplace=True)

In [ ]:
X

Save X df for animation

In [ ]:
# Save DataFrame to CSV file
animation_df = X.copy()
animation_df['Model'] =  model_df['Model'].values
animation_df.to_csv('data/animation_df.csv', index=True)

### Normalizing Features

In [ ]:
X_normalized = X.copy()

#Apply absolute value to values of docking score before normalizing
X_normalized['Docking score'] = X_normalized['Docking score'].abs()


In [ ]:
# Normalizing each column using min-max scaler
for column in X_normalized.columns:
   if X_normalized[column].max() - X_normalized[column].min() == 0:
      X_normalized[column]== X_normalized[column].max()
   else:
      X_normalized[column] = (X_normalized[column] - X_normalized[column].min()) / (X_normalized[column].max() - X_normalized[column].min()) 
   

append Model column to X_normalized

In [ ]:
# Reset the indices to match the order
df_features = model_df.copy()
df_features=df_features.reset_index(drop=True)
# df_model_reset = df_model.reset_index(drop=True)

# Append the 'model' column to the features DataFrame
X_normalized = pd.concat([X_normalized, df_features['Model']], axis=1)


In [ ]:
X_normalized

In [ ]:
X

## ANOVA F-Test for Feature Importance in Correlation Analysis

In [ ]:
from sklearn.feature_selection import f_classif


Testing: For a fixed feature, are the means significantly different across four models?

In [ ]:
# Convert the categorical target to numerical values
X_ANOVA=X_normalized.copy()
X_ANOVA['Model'] = X_ANOVA['Model'].astype('category').cat.codes

# Extract features and target
X_new = X_ANOVA.drop('Model', axis=1)
y = X_ANOVA['Model']

# Perform ANOVA F-test
f_values, p_values = f_classif(X_new, y)

# Create a DataFrame for the results
results_ANOVA = pd.DataFrame({'Feature': X_new.columns, 'F-value': f_values, 'p-value': p_values})

# Sort the results by F-value in descending order
results_ANOVA = results_ANOVA.sort_values(by='F-value', ascending=False)

In [ ]:
results_ANOVA

### Feature Variance Calculation

In [ ]:
variance = X_ANOVA.var()
print("Feature Variances:\n", variance)

## Correlation Analysis

Compute correlation matrix

In [ ]:
X_correlation=X_normalized.copy().drop('Model', axis=1)
correlation_matrix = X_correlation.corr()
X_correlation.corr().style.background_gradient(cmap='coolwarm', vmin=-1, vmax=1)

identify which features are highly correlated

In [ ]:
# Function takes as input a correlation matrix and a threshold, selects features with correlation greater or equal to
# threshol and returns 3-tuples of the form (feat1, feat2, correlation)

def get_highly_correlated_pairs(corr_matrix, threshold):
    """
    Returns pairs of features with correlation greater than or equal to the threshold.
    
    Parameters:
    corr_matrix (pd.DataFrame): Correlation matrix of features
    threshold (float): Threshold for correlation value
    
    Returns:
    List[Tuple[str, str, float]]: List of 3-tuples (feat1, feat2, correlation_score)
    """
    correlated_pairs = []
    
    # Iterate over the correlation matrix
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) >= threshold:
                feat1 = corr_matrix.columns[i]
                feat2 = corr_matrix.columns[j]
                corr_score = corr_matrix.iloc[i, j]
                correlated_pairs.append((feat1, feat2, corr_score))
                
    return correlated_pairs

We set the threshold to 0.9

In [ ]:
threshold = 0.9
highly_correlated_pairs = get_highly_correlated_pairs(correlation_matrix, threshold)
highly_correlated_pairs

Get rid of highly correlated features with the following criterion:
1. Feature is correlated with three or more features.
2. If a feature is correlated with two or one other feature, then the features with least importance in the ANOVA F-test are filtered out.

The following functions returns takes as input a list of 3-tuples of the form (feat1, feat2, corr_score) and a df with the ranked features according to ANOVA F-test. The function returns the features that must be filtered out.

In [ ]:
from collections import Counter

def filter_correlated_features(correlations, ranked_df):
    # Count the occurrences of each feature in the correlations list
    feature_counts = Counter([feat for tup in correlations for feat in tup[:2]])

    # Identify features correlated with three or more others
    high_corr_features = set([feat for feat, count in feature_counts.items() if count >= 3])

    # Identify features correlated with one or two others
    low_corr_features = [tup for tup in correlations if feature_counts[tup[0]] < 3 and feature_counts[tup[1]] < 3]

    # Filter out the least important features based on the ANOVA F-test ranking
    filtered_features = set()
    for feat1, feat2, _ in low_corr_features:
        feat1_rank = ranked_df.loc[feat1, 'ANOVA_F_Rank']
        feat2_rank = ranked_df.loc[feat2, 'ANOVA_F_Rank']
        if feat1_rank > feat2_rank:
            filtered_features.add(feat1)
        else:
            filtered_features.add(feat2)

    # Combine the high_corr_features and filtered_features to identify all features to remove
    features_to_remove = high_corr_features.union(filtered_features)
    
    return features_to_remove


Filtered features: synthetic complexity, tanimoto_sim, desc_HeavyAtomMolWt, desc_HeavyAtomCount

Create dataframe with the the least correlated features

In [ ]:
X_RF_corr_analysis = X_normalized.copy().drop(['Dice Sim', 'Synthetic Complexity', 'MolWt', 'Heavy Atom MolWt'], axis=1)

In [ ]:
X_RF_corr_analysis

## Random Forest Experiment with Correlation Analysis, Stratefied Sampling and k-Fold Cross Validation

Instead of using X_normalized, we use X_RF_corr_analysis, which is the dataframe after correlation analysis

In [ ]:
X_RF_corr_analysis.columns


In [ ]:
# Extract features and target variable
X_RF = X_RF_corr_analysis.copy().drop('Model', axis=1)
y_RF = df_features['Model']


In [ ]:
# Initialize variables for cross-validation
n_splits = 5
scores_strat = []
importances = np.zeros(X_RF.shape[1])

# Stratified K-Fold cross-validation
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

for fold, (train_index, test_index) in enumerate(skf.split(X_RF, y_RF)):
    X_train, X_test = X_RF.iloc[train_index], X_RF.iloc[test_index]
    y_train, y_test = y_RF.iloc[train_index], y_RF.iloc[test_index]
    
    # Train Random Forest model
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)
    
    # Predict and evaluate the model
    y_pred = model.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    scores_strat.append(score)
    
    # Accumulate feature importances
    importances += model.feature_importances_
    
    # Debug: Print fold number and score
    print(f'Fold {fold + 1}: Accuracy = {score}')

# Average the importances across all folds
importances /= n_splits

# Print the average prediction score
average_score = np.mean(scores_strat)
print(f'Average Prediction Score: {average_score}')

# Create a DataFrame for feature importances
feature_importance_df_strat = pd.DataFrame({
    'Feature': X_RF.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

importance_explained_rf_strat = feature_importance_df_strat['Importance'].iloc[:6].sum()
print(f'Differences Explained by Top 5 Features: {importance_explained_rf_strat}')
print(feature_importance_df_strat)

# Random Forest with Correlation Analysis, Scaffold Split and 5-fold CV

1. Extract the scaffold of each molecule using RDKit's MurckoScaffold and MakeScaffoldGeneric.
2. Create a dictionary where keys are scaffolds and values are lists of molecule indices.
3. Split the dataset into training and testing sets ensuring that molecules with the same scaffold are not split between the training and testing sets and maintaining the stratified sampling based on the molecule's model type.
4. Train a Random Forest model using the stratified sampling.

In [ ]:
df_scaffold = X_RF_corr_analysis.copy()
df_scaffold['Model']=X_normalized['Model']
df_scaffold['smiles']=molscore_df['smiles']

In [ ]:
# Make list with smiles strings
smiles_df_scaffold = df_scaffold['smiles'].tolist()

In [ ]:
# Add column with generic scaffolds
generic_scaffold_list = []
for smile in smiles_df_scaffold:
    scaffold = Chem.MolToSmiles(MurckoScaffold.MakeScaffoldGeneric(Chem.MolFromSmiles(smile)))
    generic_scaffold_list += [scaffold]

df_scaffold['Generic_Scaffold'] = generic_scaffold_list


In [ ]:
df_scaffold

In [ ]:
# Separate features and target
X_scaff = df_scaffold.drop(columns=['Model', 'Generic_Scaffold', 'smiles'])
y_scaff = df_scaffold['Model']
scaffold = df_scaffold['Generic_Scaffold']

In [ ]:
# Create a dictionary to map scaffolds to indices
scaffold_dict = defaultdict(list)
for idx, s in enumerate(scaffold):
    scaffold_dict[s].append(idx)

# Create a dictionary to map scaffolds to their labels
scaffold_labels = {}
for s, indices in scaffold_dict.items():
    scaffold_labels[s] = y_scaff.iloc[indices].tolist()


The following function ensures:
1. Repeated scaffolds are not added to both train and test sets
2. Each split has an even class distribution, where the each class is the model type


In [ ]:
# Define a function for stratified splitting by scaffold
def stratified_scaffold_split(X, y, scaffold_dict, scaffold_labels, n_splits=5, random_state=None):
    np.random.seed(random_state)
    scaffolds = list(scaffold_dict.keys())
    np.random.shuffle(scaffolds)
    fold_size = len(scaffolds) // n_splits
    
    for i in range(n_splits):
        test_scaffolds = set(scaffolds[i * fold_size: (i + 1) * fold_size])
        train_scaffolds = set(scaffolds) - test_scaffolds
        
        train_indices = [idx for s in train_scaffolds for idx in scaffold_dict[s]]
        test_indices = [idx for s in test_scaffolds for idx in scaffold_dict[s]]
        
        # Ensure the stratified split maintains class proportions within scaffolds
        y_train = y.iloc[train_indices]
        y_test = y.iloc[test_indices]
        
        if len(set(y_test)) > 0 and len(set(y_train)) > 0:
            yield train_indices, test_indices


Run random forest

In [ ]:
# Initialize variables for cross-validation
n_splits = 5
scores_scaffold = []
importances = np.zeros(X_scaff.shape[1])  # Initialize with the number of features in X_scaff

# Perform 5-fold cross-validation
valid_splits = 0
for fold, (train_index, test_index) in enumerate(stratified_scaffold_split(X_scaff, y_scaff, scaffold_dict, scaffold_labels, n_splits=n_splits, random_state=42)):
    valid_splits += 1
    X_train, X_test = X_scaff.iloc[train_index], X_scaff.iloc[test_index]
    y_train, y_test = y_scaff.iloc[train_index], y_scaff.iloc[test_index]
    
    # Train Random Forest model
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)
    
    # Predict and evaluate the model
    y_pred = model.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    scores_scaffold.append(score)
    
    # Check if the feature importances match the expected shape
    if len(model.feature_importances_) == importances.shape[0]:
        # Accumulate feature importances
        importances += model.feature_importances_
    else:
        raise ValueError(f"Shape mismatch: expected {importances.shape[0]} but got {len(model.feature_importances_)}")
    
    # Debug: Print fold number and score
    print(f'Fold {fold + 1}: Accuracy = {score}')

# Check if we got valid splits
if valid_splits == 0:
    raise ValueError("No valid train-test splits found. Please check the splitting function.")

# Average the feature importances
importances /= valid_splits

# Create a DataFrame for feature importances
feature_importance_scaffold_df = pd.DataFrame({
    'Feature': X_scaff.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# Calculate the cumulative importance explained by the top 5 features
importance_explained_rf_scaffold = feature_importance_scaffold_df['Importance'].iloc[:5].sum()

# Print the results
print(f'Prediction Scores: {scores_scaffold}')
print(f'Average Prediction Score: {np.mean(scores_scaffold)}')
print(f'Differences Explained by Top 5 Features: {importance_explained_rf_scaffold}')
print(f'Feature Importances:\n{feature_importance_scaffold_df}')


Comparing Random Forest Experiments

### Visualizations

In [ ]:
feature_importance_scaffold = feature_importance_scaffold_df.head(10)
average_pred_scaffold = np.mean(scores_scaffold)
feature_importance_strat = feature_importance_df_strat.head(10)
average_pred_strat = np.mean(scores_strat)


In [ ]:
from matplotlib.lines import Line2D
# List of dataframes and titles
dataframes = [feature_importance_scaffold, feature_importance_strat]
average_prediction = [average_pred_scaffold , average_pred_strat]
top_6_explain = [importance_explained_rf_scaffold, importance_explained_rf_strat]

titles = ['Feature Importances with Scaffold Stratified Sampling and 5-Fold CV',
           'Feature Importances with Stratified Sampling and 5-Fold CV']

# Calculate the max y-value for setting the same y-axis scale
max_value = max([df['Importance'].max() for df in dataframes]) + 0.01
min_value = min([df['Importance'].min() for df in dataframes]) - 0.005

# Create a figure and axis objects with subplots
fig, axs = plt.subplots(1, 2, figsize=(20, 11))
fig.suptitle('Random Forest Experiments Comparison',fontsize=16, fontweight='bold', y=1.00)

# Flatten the axes array for easy iteration
axs = axs.flatten()

# Iterate over dataframes and axes
for i, (df, title) in enumerate(zip(dataframes, titles)):
    sns.barplot(x='Feature', y='Importance', palette='colorblind', data=df, ax=axs[i])
    axs[i].set_ylim(min_value, max_value)  # Set the same y-axis scale
    axs[i].tick_params(axis='x', rotation=90)  # Rotate x-ticks
    axs[i].set_title(title)

    # Add custom legend with average prediction and cumulative weight of top 5 features
    ave_pred = average_prediction[i] * 100
    top_6_expl = top_6_explain[i]

    # Create custom legend lines
    legend_elements = [
        Line2D([0], [0], color='w', label=f'Avg. Prediction: {ave_pred:.3f} %'),
        Line2D([0], [0], color='w', label=f'Cumulative Weight Top 6: {top_6_expl:.3f}')
    ]

    # Add legend to the plot
    axs[i].legend(handles=legend_elements, loc='upper right', frameon=False)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

# Sensitivity Analysis with RF Correlation Analysis, Scaffold Split and 5-fold CV

The next natural question to ask is: how sensible are these results (feature's weights) with respect to the input parameters (initial distributions)

We evaluate how sensitive the feature importances and model performance are by training the model on different bootstrap samples of the data. For this analysis, we use the dataframe after correlation analysis.

In [ ]:
# Extract features and target variable
X_sen_ana = X_RF_corr_analysis.copy().drop('Model', axis=1)
y = df_features['Model']

In [ ]:
from sklearn.utils import resample

# Define number of bootstraps and number of splits for cross-validation
num_bootstraps = 100
num_splits = 5

# Initialize storage for feature importances and rankings
importances = np.zeros((num_bootstraps, X_sen_ana.shape[1]))
all_predictions = []

# Stratified K-Fold cross-validation
skf = StratifiedKFold(n_splits=num_splits)


In [ ]:
for i in range(num_bootstraps):
    X_resample, y_resample = resample(X_sen_ana, y, random_state=i)

    fold_importances = np.zeros((num_splits, X_sen_ana.shape[1]))
    fold_predictions = []

    valid_splits = 0
    for fold_idx, (train_idx, test_idx) in enumerate(stratified_scaffold_split(X_resample, y_resample, scaffold_dict, scaffold_labels, n_splits=num_splits, random_state=42)):
        valid_splits += 1
        X_train, X_test = X_resample.iloc[train_idx], X_resample.iloc[test_idx]
        y_train, y_test = y_resample.iloc[train_idx], y_resample.iloc[test_idx]

        rf = RandomForestClassifier(n_estimators=250, max_depth=None, random_state=1)
        rf.fit(X_train, y_train)

        y_pred = rf.predict(X_test)
        fold_predictions.append(accuracy_score(y_test, y_pred))

        fold_importances[fold_idx, :] = rf.feature_importances_

    if valid_splits == 0:
        raise ValueError("No valid train-test splits found. Please check the splitting function.")

    importances[i, :] = np.mean(fold_importances, axis=0)
    all_predictions.append(np.mean(fold_predictions))

# Calculate mean and standard deviation of importances
mean_importances = np.mean(importances, axis=0)
std_importances = np.std(importances, axis=0)

# Calculate mean and standard deviation of predictions
mean_prediction = np.mean(all_predictions)
std_prediction = np.std(all_predictions)

print("Mean Importances:\n", mean_importances)
print("Standard Deviation of Importances:\n", std_importances)
print("Mean Prediction:\n", mean_prediction)
print("Standard Deviation of Prediction:\n", std_prediction)

### Visualizations

In [ ]:
from matplotlib.lines import Line2D
# Create a DataFrame for feature importances
feature_importance_df = pd.DataFrame({
    'Feature': X_sen_ana.columns,
    'Mean Importance': mean_importances,
    'Std Importance': std_importances
}).sort_values(by='Mean Importance', ascending=False)

# Select top 10 features
top_10_features = feature_importance_df.head(10)
# Create a figure and axis objects with subplots
fig, ax = plt.subplots(figsize=(12, 8))

# Plot the histogram
sns.set_palette("colorblind")
bars = ax.bar(top_10_features['Feature'], top_10_features['Mean Importance'], yerr=top_10_features['Std Importance'], capsize=5, color=sns.color_palette("colorblind"))

# Add labels and title
ax.set_xlabel('Feature')
ax.set_ylabel('Average Importance')
ax.set_title('Average Top 10 Feature Importances', fontsize=16, fontweight='bold',  y=1.05)
ax.set_xticklabels(top_10_features['Feature'], rotation=90)

# Create legend elements
legend_elements = [
    Line2D([0], [0], color='w', markersize=10, label=f'Avg. Prediction: {mean_prediction*100:.3f} %' ),
    Line2D([0], [0], color='w', markersize=10, label=f'STD of Prediction: {std_prediction*100:.3f} %'),
    Line2D([0], [0], color='w', markersize=10, label=f'Explained by Top 5:  62.409 %')
]

# Add legend to the plot
ax.legend(handles=legend_elements, loc='upper right', frameon=False)

# Display the plot
plt.show()

print("Mean Importances:\n", mean_importances)
print("Standard Deviation of Importances:\n", std_importances)


In [ ]:
X_RF_corr_analysis

### Visualizing top distributions

Closer look at distributions of the 10 features with highest average weight

In [ ]:
std_importances[0]

In [ ]:
# Get indices of the top 10 features
top_10_indices = np.argsort(mean_importances)[-10:][::-1]

# Extract importances and rankings for top 10 features
top_10_importances = importances[:, top_10_indices]
top_10_rankings = rankings[:, top_10_indices]

top_10_indices

In [ ]:
mean_importances
X_sen_ana.columns[13]

In [ ]:
# Determine the max x and y limits for the histograms
max_importance = np.max(top_10_importances)
max_frequency = 0

min_importance = np.min(top_10_importances)

for i in range(10):
    counts, _ = np.histogram(top_10_importances[:, i], bins=30)
    max_frequency = max(max_frequency, np.max(counts))


In [ ]:
from matplotlib.lines import Line2D

plt.figure(figsize=(20, 9))

for i in range(10):
    mean = mean_importances[top_10_indices[i]]
    std = std_importances[top_10_indices[i]]
    
    ax = plt.subplot(2, 5, i + 1)
    # Plot histogram
    ax.hist(top_10_importances[:, i], bins=30, edgecolor='k', alpha=0.7)
    
    # Add title and legend with custom lines and labels
    ax.set_title(f'{X_sen_ana.columns[top_10_indices[i]]} Importance', pad=20)
    ax.set_xlabel('Importance')
    ax.set_ylabel('Frequency')
    ax.set_xlim(min_importance - 0.01, max_importance + 0.01)
    ax.set_ylim(0, max_frequency + 2)

    # Create empty handles for legend
    mean_handle = Line2D([0], [0], color='none', marker='o', markerfacecolor='black', markersize=6, linestyle='None')
    std_handle = Line2D([0], [0], color='none', marker='o', markerfacecolor='black', markersize=6, linestyle='None')
    
      # Add the legend
    ax.legend(handles=[mean_handle, std_handle], labels=[
        f'Mean: {mean:.3f}', 
        f'Std: {std:.3f}'
    ], loc='upper left')

plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust layout to fit titles and labels
plt.suptitle('Distribution of Feature Importance for Top 10 Features', fontsize=16, fontweight='bold', y=1.00)
plt.show()

We implement Silhouette score to find the optimal subset of three features that are best at clustering the four models

# Optimal Clustering Analysis

In [ ]:
import itertools
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation

Load data for Optimal Clustering Analysis and create df with top 10 features

In [ ]:
DF_FILEPATH = f'data/animation_df.csv'
X =  pd.read_csv(DF_FILEPATH, index_col=0)


In [ ]:
df_cluster = X[['CLogP', 'Heavy Atom Count', '# Rotatable Bonds', 'Formal Charge',
        'Tanimoto Sim', 'Docking score', 'Max Consecutive Rotatable Bonds', 'ifp_intersection', 'TPSA', 'Synthetic Accessibility', 'Model']]

### Optimal Clustering Analysis

In [ ]:
le = LabelEncoder()
df_cluster['model_encoded'] = le.fit_transform(df_cluster['Model'])

In [ ]:
# Get feature columns
feature_columns = df_cluster.columns.difference(['Model', 'model_encoded'])

# Generate all combinations of three features
combinations = list(itertools.combinations(feature_columns, 3))

The following functions computes the combination of three features with the highest Silhouette Score

In [ ]:
def evaluate_feature_combinations(df, combinations):
    best_score = -1
    best_combination = None
    
    for combo in combinations:
        # Compute silhouette score
        X = df[list(combo)]
        try:
            score = silhouette_score(X, df['model_encoded'], metric='euclidean')
        except ValueError:
            score = -1  # If not enough samples per cluster

        print(f"Combination: {combo}, Silhouette Score: {score:.2f}")

        if score > best_score:
            best_score = score
            best_combination = combo
    
    return best_combination, best_score

# Evaluate combinations
best_comb, best_score = evaluate_feature_combinations(df_cluster, combinations)
print(f"Best Combination: {best_comb}, Silhouette Score: {best_score:.2f}")
